In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,AveragePooling2D,Flatten
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint



In [9]:
(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()

X_train,X_test=X_train/255,X_test/255
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)

169009152/169001437 [==============================] - 41s 0us/step


In [11]:
aug_data=ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.3)
aug_data.fit(X_train)

In [12]:
shape=(32,32,3)

inputs = keras.Input(shape=shape)

a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(inputs)  #1
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)   #2
a_temp=a
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)#\3
a=layers.Conv2D(32,(3,3),strides=1,padding="same")(a)#4
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #1
                               
#a_temp=a
##########################################################################
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #5
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #6
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #7
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #8
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #2

###############################################################################

a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #9
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #10
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #11
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #12
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #3

#################################################################################

a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #13
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #14
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #15
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #16
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
############################################################3
a=layers.AveragePooling2D(pool_size=(2,2),strides=1,padding="same")(a)

a=Flatten()(a)

#############################################################

a=layers.Dense(512,activation="relu")(a)
outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)


model = keras.Model(inputs, outputs)

In [16]:
adam = Adam(learning_rate= 0.0001,clipvalue=0.6)


model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])
call=EarlyStopping(monitor="val_loss",verbose=1,mode="auto",patience=5,restore_best_weights=True)
checkpoint=ModelCheckpoint('Resnet_adam_without_regularization',monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=True,model='auto',period=1)


In [17]:
model.fit_generator(aug_data.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)/128,epochs=80,validation_data=(X_test,y_test),verbose=1,callbacks=[call,checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/80
391/390 [==============================] - ETA: 0s - loss: 3.9650 - accuracy: 0.0989
Epoch 00001: val_accuracy improved from -inf to 0.18880, saving model to sgd_non_reg_VGG16_weights.hdf5
391/390 [==============================] - 200s 511ms/step - loss: 3.9650 - accuracy: 0.0989 - val_loss: 3.4381 - val_accuracy: 0.1888
Epoch 2/80
391/390 [==============================] - ETA: 0s - loss: 3.3755 - accuracy: 0.1950
Epoch 00002: val_accuracy improved from 0.18880 to 0.24760, saving model to sgd_non_reg_VGG16_weights.hdf5
391/390 [==============================] - 200s 511ms/step - loss: 3.3755 - accuracy: 0.1950 - val_loss: 3.1222 - val_accuracy: 0.2476
Epoch 3/80
391/390 [==============================] - ETA: 0s - loss: 3.0767 - accuracy: 0.2486
Epoch 00003: val_accuracy improved from 0.24760 to 0.29070, saving model to sgd_non_reg_VGG16_weights.hdf5
391/390 [==============================] - 200s 

In [18]:
y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

In [19]:
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print(accuracy_score(Y_test,y_pred))
print(precision_score(Y_test,y_pred,average="weighted"))
print(recall_score(Y_test,y_pred,average="weighted"))

              precision    recall  f1-score   support

           0       0.86      0.65      0.74       100
           1       0.55      0.67      0.60       100
           2       0.61      0.38      0.47       100
           3       0.33      0.30      0.31       100
           4       0.43      0.29      0.35       100
           5       0.39      0.56      0.46       100
           6       0.62      0.61      0.61       100
           7       0.48      0.55      0.51       100
           8       0.56      0.60      0.58       100
           9       0.74      0.55      0.63       100
          10       0.28      0.39      0.33       100
          11       0.36      0.34      0.35       100
          12       0.45      0.61      0.51       100
          13       0.47      0.43      0.45       100
          14       0.46      0.46      0.46       100
          15       0.36      0.43      0.39       100
          16       0.66      0.50      0.57       100
          17       0.70    

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,AveragePooling2D,Flatten
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()

X_train,X_test=X_train/255,X_test/255
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)

shape=(32,32,3)

inputs = keras.Input(shape=shape)

a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(inputs)  #1
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)   #2
a_temp=a
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)#\3
a=layers.Conv2D(32,(3,3),strides=1,padding="same")(a)#4
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #1
                               
#a_temp=a
##########################################################################
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #5
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #6
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #7
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #8
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #2

###############################################################################

a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #9
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #10
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #11
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #12
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #3

#################################################################################

a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #13
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #14
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #15
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #16
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
############################################################3
a=layers.AveragePooling2D(pool_size=(2,2),strides=1,padding="same")(a)

a=Flatten()(a)

#############################################################

a=layers.Dense(512,activation="relu")(a)
outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)


model = keras.Model(inputs, outputs)

model.load_weights('../weights/Resnet_adam_without_regularization.hdf5')


y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("the accuracy obtained is {}".format(accuracy_score(Y_test,y_pred)))
print("precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("the recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
              precision    recall  f1-score   support

           0       0.78      0.80      0.79       100
           1       0.68      0.63      0.66       100
           2       0.57      0.44      0.50       100
           3       0.29      0.12      0.17       100
           4       0.41      0.39      0.40       100
           5       0.53      0.41      0.46       100
           6       0.72      0.66      0.69       100
           7       0.57      0.52      0.54       100
           8       0.62      0.70      0.66       100
           9       0.69      0.61      0.65       100
          10       0.31      0.44      0.37       100
          11       0.41      0.32      0.36       100
          12       0.57      0.50      0.53       100
          13       0.49      0.33      0.39       100
          14       0.38      0.60      0.47       100
          15     